In [559]:
from aocd import get_data

YEAR = 2022
DAY = 11

dat = get_data(day=DAY,year=YEAR)

with open('input/11','w') as f:
    f.write(dat)

f.close()

In [560]:
with open('input/11') as f:
    dat = f.read().split('\n')

In [561]:
from math import floor
import numpy as np
class Monkey:
    def __init__(self, id):
        self.id = id
        self.items = []
        self.items2 = []
        self.testDivNumber = 1
        self.opOperand = '' # Either '*' or '+'
        self.opTarget = 'old'#either 'old' or 'int'
        self.trueTarget = 0
        self.falseTarget = 0
        self.inspected = 0

    def __repr__(self) -> str:
        out = 'Monkey ' + str(self.id) + ': ' + ', '.join([str(i) for i in self.items])
        return out

    

    def play(self):
        out = []
        self.inspected += len(self.items)
        for i in self.items:
            if(self.opOperand == '+'):
                tmp = floor((i + i if self.opTarget == 'old' else i + int(self.opTarget)) / 3)
            else:
                tmp = floor((i * i if self.opTarget == 'old' else i * int(self.opTarget)) / 3)
            newTarget = (self.trueTarget, tmp) if (tmp % self.testDivNumber == 0) else (self.falseTarget, tmp)
            out.append(newTarget)
        self.items = []
        return out

    def initPlay2(self,mods):
        self.items2 = [[i%m for m in mods]  for i in self.items]

    def play2(self,mods):
        out = []
        # print(self.items2)
        # print(f'length of items: {len(self.items2)}, type of items2: {type(self.items2)}')
        self.inspected += len(self.items2)
        # print(f'out: {out}')
        for i in self.items2: # i is np list [, , , ]
            # print(f'current item: {i}, type: {type(i)}')
            if(self.opOperand == '+'):   
                # print('plus')              
                tmp = np.array([( (i[j] + i[j]) if self.opTarget == 'old' else (i[j] + int(self.opTarget)))%mods[j] for j in range(len(i))])
            else:
                # print('mult')
                
                tmp = np.array([( (i[j] * i[j]) if self.opTarget == 'old' else (i[j] * int(self.opTarget)))%mods[j] for j in range(len(i))])
                # print(f'tmp: {tmp}, type: {type(tmp)}')
            newTarget = (self.trueTarget, tmp) if (tmp[self.id] == 0) else (self.falseTarget, tmp)
            # print(f'newTarget: {newTarget}')
            out.append(newTarget)
            # print(out)
        self.items2 = []
        # print(f'out: {out}')
        return out
    


In [562]:
def initMonkeys():
    monkeys = []
    mods = [] #for part 2
    # Parse
    idx =0
    import regex as re
    while idx < len(dat):
        id = int(re.findall('Monkey (\d*):',dat[idx])[0])
        monkeys.append(Monkey(id))
        items = re.findall(' (\d+)',dat[idx+1],overlapped=False)
        for o in items:
            monkeys[-1].items.append(int(o))
        monkeys[-1].opOperand = re.findall('= old (\*|\+)',dat[idx+2])[0]
        monkeys[-1].opTarget = re.findall('(?:\*|\+) ([0-9]+|old)',dat[idx+2])[0]
        monkeys[-1].testDivNumber = int(re.findall('([0-9]+)',dat[idx+3])[0])
        monkeys[-1].trueTarget = int(re.findall('([0-9]+)',dat[idx+4])[0])
        monkeys[-1].falseTarget = int(re.findall('([0-9]+)',dat[idx+5])[0])
        idx = idx+7
    return monkeys


In [563]:
def playRound(monkeys):
    for m in monkeys:
        out = m.play()
        for o in out:
            monkeys[o[0]].items.append(o[1])


def printMonkeys(monkeys):
    for m in monkeys:
        print(m)
    print('\n')


In [564]:
# print('initial:')
# printMonkeys(monkeys)
monkeys = initMonkeys()
for i in range(20):
    playRound(monkeys)
    # printMonkeys(monkeys)

for m in monkeys:
    print(f'inspected: {m.inspected}')

print(f'Problem 1: {sorted([m.inspected for m in monkeys],reverse=True)[0]*sorted([m.inspected for m in monkeys],reverse=True)[1]}')


inspected: 343
inspected: 333
inspected: 347
inspected: 22
inspected: 22
inspected: 350
inspected: 44
inspected: 320
Problem 1: 121450


In [565]:
def playRound2(monkeys):
    output  = ''
    for i,m in enumerate(monkeys):
        # print(f'\nPlaying monkey {i}: \n')
        out = m.play2(mods)
        output += str(out) + '\n'
        for o in out:
            monkeys[o[0]].items2.append(o[1])
    # return output


In [566]:
# For problem 2 a different structure in needed.
#instead of listing items, we list a list with all 
# modulos for the divisiblity rules
# 79 becomes (79%23, 79%19, 79%13, 79%17)

monkeys = initMonkeys()
mods = [m.testDivNumber for m in monkeys]

for m in monkeys:
    m.initPlay2(mods)

for i in range(10000):
    playRound2(monkeys)

for m in monkeys:
    print(f'inspected: {m.inspected}')

print(f'Problem 2: {sorted([m.inspected for m in monkeys],reverse=True)[0]*sorted([m.inspected for m in monkeys],reverse=True)[1]}')



inspected: 165766
inspected: 161105
inspected: 159985
inspected: 9209
inspected: 18375
inspected: 166922
inspected: 13590
inspected: 169205
Problem 2: 28244037010
